# Transmission between Antennas

![Experiment](transmission/experiment.jpg)

In this experiment, a stationary wave appears between transmitting and receiving antennas.

By slowly varying their distance and mapping this wave, it is possible to find the frequency of the transmitted signal.

**Objectvie:** find the frequency of the transmitted signal.

In [ ]:
# Import necessary modules
import numpy
from matplotlib import pyplot

# Only in Jupyter Notebooks
%matplotlib notebook

The experimental data is in a comma-separated values (CSV) file.

The *magic* command `%cat` inside the Jupyter Notebook (or IPython REPL) shows the contents of a file.

In [ ]:
%cat transmission/data.csv

The data can be read manually and stored in a list of lists of floats:

```python
data = []
with open('transmissao/dados.csv') as infile:
    print(infile.readline())
    for line in infile:
        data.append([float(i) for i in line.split(';')])
data = numpy.array(data)
```

Alternatively, the `csv` built-in module contains the `reader` function to automate the process and take care of more general data.

In this case, a third option is more indicated because the file contains only nummerical data that will be later manipulated: the function `loadtxt` from `numpy`.
It will be faster, simpler and it will provide the data in a `numpy` array ready for vector manipulations.

In [ ]:
numpy.loadtxt?

In [ ]:
data = numpy.loadtxt('transmission/data.csv', delimiter=';', skiprows=1)
data

In [ ]:
data.shape

In [ ]:
data[0, :] # View row 0 (first), all columns

In [ ]:
data[0, :].shape

The `x` values can be extracted from the first column of the `data` array; all remaining columns contain the measurements `v`.

**Be careful:** `x` does not copy the values from `data`, it is a *view* of them, therefore changing an element in `x` will also affect `data`.

In [ ]:
x = data[:, 0]
x

In [ ]:
v = data[:, 1:]
v

In [ ]:
fig, ax = pyplot.subplots(1, 1)

ax.plot(x, v[:, 0], '.')
ax.plot(x, v[:, 1], '.')
ax.plot(x, v[:, 2], '.')
ax.plot(x, v[:, 3], '.')

If the means and standard deviations from the 4 experiment runs are required, they can be easily calculated:

In [ ]:
v0 = v[0, :] # First row of the measurement data: 4 measurements taken at x[0]
print(v0, numpy.mean(v0), numpy.std(v0))

In [ ]:
%%timeit
y = numpy.zeros(len(x))
for i in range(len(x)):
    y[i] = numpy.mean(v[i, :])

In [ ]:
%%timeit
y = numpy.mean(v, axis=1)

In [ ]:
y = numpy.mean(v, axis=1)
e = numpy.std(v, axis=1)
y

In [ ]:
fig, ax = pyplot.subplots(1, 1)

ax.errorbar(x, y, e, capsize=3)

ax.set_xlabel('Distance (mm)')
ax.set_ylabel('Signal (normalized)')

ax.grid()

The spacial period of the standing wave $\ell$ is directly related to the frequency $f$ and wavelength $\lambda$ of the transmitted signal:

\begin{align*}
\lambda &= 2\ell \\
f &= \frac{c}{\lambda} = \frac{c}{2\ell}
\end{align*}

There are several ways of finding $\ell$ from the available data.
Due to the sparsity of points and existing uncertainties, a good solution is to use a model of the experiment to fit the data.
That is preferable to simply finding the distance between consecutive peaks and valeys or using a Discrete Fourier Transform to find the main spacial frequency of the data, because it introduces information about the data that is not present in the measurements.

In the previous notebook a linear fit was enough to model the data, but now the model is, in a first approximation, a sinusoidal curve.
A non-linear fitting method is required:

In [ ]:
from scipy.optimize import curve_fit

curve_fit?

Nosso modelo deve ser uma função da posição e dos parâmetros que queremos otimizar.
Por se tratar de um problema não-linear, é preciso dar um chute inicial razoável para o algoritmo de ajuste.

In [ ]:
def model(x, l, a, x0, y0):
    return y0 + a * numpy.sin(2*numpy.pi/l*(x - x0))

model(0.25, 5, 2, 0, 0)

In [ ]:
x_fit = numpy.array([0, 0.25, 0.5, 0.75, 1])
model(x_fit, 10, 1, 0, 0)

In [ ]:
args = [10, 1, 0, 0]
model(x_fit, *args)

In [ ]:
init = (15, 0.2, 291, 0.75)

x_fit = numpy.linspace(x[0], x[-1], 201)

y_init = model(x_fit, *init)

fig, ax = pyplot.subplots(1, 1)

ax.errorbar(x, y, e, capsize=3, ls='', marker='.')
ax.plot(x_fit, y_init)

In [ ]:
opt, cov = curve_fit(model, x, y, init, sigma=e)

print(opt)
print(numpy.sqrt(numpy.diag(cov)))

In [ ]:
y_fit = model(x_fit, *opt)

fig, ax = pyplot.subplots(1, 1)

ax.errorbar(x, y, e, capsize=3, ls='', marker='.')
ax.plot(x_fit, y_fit)

ax.set_xlabel('Distance (mm)')
ax.set_ylabel('Signal (normalized)')

ax.grid()

Conhecendo o modelo, sabemos qua a distância entre máximos é a metade do comprimento de onda do sinal transmitido.

In [ ]:
f = 3e8 / (2 * opt[0] * 1e-3)
print(f'Frequency: {f * 1e-9:.1f} GHz')

## Exercise

Modify the model to accept the transmitted signal frequency $f$ instead of the the standing wave period $\ell$ as an argument.

### Bonus

Find the uncertainty of the frequency based on the model fitting from the covariance matrix.

In [ ]:
def model2(x, f, a, x0, y0):
    ...

## Appendix

An even more accurate model is presented below.
It models the multiple reflections at both antenas and losses.

In [ ]:
def model3(x, f, a, e0, g, p0, p1):
    phi = (a + 6.67e-3j * numpy.pi * f) * (x - 280)
    y = e0 * (numpy.exp(1j * p0 - phi) + g * numpy.exp(1j * p1 + phi))
    return numpy.abs(y)**2

init3 = (10, 0.001, 0.9, 0.1, 0, 0)
opt3, cov3 = curve_fit(model3, x, y, init3)

y_init = model3(x_fit, *init3)
y_fit = model3(x_fit, *opt3)

fig, ax = pyplot.subplots(2, 1, figsize=(6, 6))

for a in ax:
    a.errorbar(x, y, e, capsize=3, ls='', marker='.')
    a.set_ylabel('Signal (normalized)')
    a.grid()
ax[1].set_xlabel('Distance (mm)')

ax[0].plot(x_fit, y_init)
ax[1].plot(x_fit, y_fit)

print('Frequency: ({:.2f} ± {:.2f}) GHz'.format(opt3[0], cov3[0, 0]**0.5))